<a href="https://colab.research.google.com/github/EdgarAgOr/FAP_Cheminformatic_PET/blob/main/3_Data_Curation_protocol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv ('/content/Bioactividad_FAPBSITE_preprocesada_data.csv')

In [ ]:
df

In [ ]:
import pandas as pd
! pip install -q rdkit
! pip install -q molvs

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from molvs.standardize import Standardizer
from molvs.charge import Uncharger, Reionizer
from molvs.fragment import LargestFragmentChooser
from molvs.tautomer import TautomerCanonicalizer
from rdkit.Chem.rdmolops import GetFormalCharge, RemoveStereochemistry

In [ ]:
df.head(10)

In [ ]:
STD = Standardizer ()
LFC = LargestFragmentChooser ()
UC = Uncharger ()
RI = Reionizer()
TC = TautomerCanonicalizer()

In [ ]:
def pretreatment (notations):
    try:
        mol=Chem.MolFromSmiles(notations)
        if mol == None:
            return "Error 1"
        else:
            mol = STD (mol)
            mol = LFC(mol)
            allowed_elements = {"H","B","C","N","O","F","Si","P","S","Cl","Br","I"}
            actual_elements = set([atom.GetSymbol()for atom in mol.GetAtom()])
            if len (actual_elements-allowed_elements)==0:
                mol = UC(mol)
                mol = RI(mol)
                RemoveStereochemistry(mol)
                mol = TC(mol)
                return Chem.MolToSmiles(mol)
            else:
                return "Error 2"
    except:
        return "Nope"

In [ ]:
df ['Smiles standarization'] = [pretreatment (i) for i in df ['Smiles']]

In [ ]:
df

In [ ]:
df = df[df['Smiles']!= 'Error 1']

In [ ]:
df = df[df['Smiles'] !='Error 2']

In [ ]:
df = df [df['Smiles']!="Nope"].reset_index(drop= True)

In [ ]:
df

In [ ]:
df = df.drop_duplicates(subset=['Smiles'], keep = 'first')

In [ ]:
df = df.drop_duplicates(subset=['BindingDB_id'], keep = 'first')

In [ ]:
df

In [ ]:
df = df.dropna()

df

In [ ]:
df.describe()

In [ ]:
def pIC50(input):
    pIC50 = []
    for i in input["IC50_nM"]:
        molar = i*(10**-9)
        pIC50.append(-np.log10(molar))
    input["pIC50"]=pIC50
    x = input.drop("pIC50",T)

    return x

In [ ]:
df.describe()

In [ ]:
df

In [ ]:
import numpy as np

In [ ]:
df_final = pIC50(df)

In [ ]:
df_final

In [ ]:
df_final.describe()

In [ ]:
df.to_csv ("Bioactividad_FAPBSITE_curada_data.csv", index = False)